In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

%matplotlib inline

In [166]:
#url = 'http://www.boxofficemojo.com/movies/?page=intl&id=marvel2016.htm'
url = 'http://www.boxofficemojo.com/movies/?page=intl&id=disney2016.htm'

resp = requests.get(url)
page = resp.text
soup = BeautifulSoup(page)

C:\Python34\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [167]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-type">
<title>Zootopia (2016) - International Box Office Results - Box Office Mojo</title>
<style type="text/css">
table.chart-wide { width: 100%; }
</style>
<meta content="zootopia, movie, film, international, box office, gross, foreign, overseas, world wide, worldwide, country, by country, result, release date, release, schedule, chart, revenue, box office mojo" name="keywords">
<meta content="Zootopia foreign box office results and release schedule." name="description">
<link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css">
<link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css">
<script src="/js/jscharts.js" type="text/javascript"></script>
</link></link></meta></meta></meta></head>
<body>


In [168]:
topTable = soup.find_all('table')

for i in range(len(topTable)):
    print(len(topTable[i]))

6
2
2
2
1
4
1
122


In [169]:
toptopTable = topTable[1].find_all('td')
type(toptopTable)

bs4.element.ResultSet

In [170]:
box = soup.find_all('table')

In [171]:
topBox = box[4]
topBox

<table border="0" cellpadding="0" cellspacing="0"><tr><td>
<center><table bgcolor="#dcdcdc" border="0" cellpadding="4" cellspacing="1" width="95%"><tr bgcolor="#ffffff"><td align="center" colspan="2"><font size="4"><font size="3">Domestic Total as of Jul. 14, 2016:</font> <b>$341,072,546</b></font></td></tr><tr bgcolor="#ffffff"><td valign="top">Distributor: <b><a href="/studio/chart/?studio=buenavista.htm">Buena Vista</a></b></td><td valign="top">Release Date: <b><nobr><a href="/schedule/?view=bydate&amp;release=theatrical&amp;date=2016-03-04&amp;p=.htm">March 4, 2016</a></nobr></b></td></tr><tr bgcolor="#ffffff"><td valign="top">Genre: <b>Animation</b></td><td valign="top">Runtime: <b>1 hrs. 48 min.</b></td></tr><tr bgcolor="#ffffff"><td valign="top">MPAA Rating: <b>PG</b></td><td valign="top">Production Budget: <b>N/A</b></td></tr></table> </center></td>
</tr></table>

## Here is section where we get top box information

In [172]:
for element in topBox.find_all('td'):
    print(element.text)
    print('----------')


Domestic Total as of Jul. 14, 2016: $341,072,546Distributor: Buena VistaRelease Date: March 4, 2016Genre: AnimationRuntime: 1 hrs. 48 min.MPAA Rating: PGProduction Budget: N/A 
----------
Domestic Total as of Jul. 14, 2016: $341,072,546
----------
Distributor: Buena Vista
----------
Release Date: March 4, 2016
----------
Genre: Animation
----------
Runtime: 1 hrs. 48 min.
----------
MPAA Rating: PG
----------
Production Budget: N/A
----------


In [173]:
def findRelevantData(tableElement):
    listOfTableData = tableElement.find_all('td')
    distributorText = listOfTableData[2].text.split(':')[1].strip()
    
    # convert to actual number
    prodBudget = listOfTableData[-1].text.split(':')[1].strip()
    mpaa_rating = listOfTableData[-2].text.split(':')[1].strip()
    
    # convert this to datetime
    releaseDate = listOfTableData[3].text.split(':')[1].strip()
    
    runTime = listOfTableData[-3].text.split(':')[1].strip()
    genre = listOfTableData[4].text.split(':')[1].strip()
    
    return distributorText, prodBudget, mpaa_rating, releaseDate, runTime, genre

print(findRelevantData(topBox))

('Buena Vista', 'N/A', 'PG', 'March 4, 2016', '1 hrs. 48 min.', 'Animation')


## Country and revenue data

In [174]:
countryData = box[7]
countryData

<table border="0" cellpadding="3" cellspacing="1"><tr bgcolor="#dcdcdc"><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=disney2016.htm&amp;sort=country&amp;order=ASC&amp;p=.htm">Country<br>(click to view weekend breakdown)</br></a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=disney2016.htm&amp;sort=studio&amp;order=ASC&amp;p=.htm">Dist.</a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=disney2016.htm&amp;sort=ReleaseDate&amp;order=DESC&amp;p=.htm">Release<br>Date</br></a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=disney2016.htm&amp;sort=opengross&amp;order=DESC&amp;p=.htm">Opening<br>Wknd</br></a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=disney2016.htm&amp;sort=percoftotal&amp;order=DESC&amp;p=.htm">% of<br>Total</br></a></font></td><td align="center" colspan="2"><font size="2"><a href="/movies/?page=intl&amp;id=disney20

In [175]:
countryDataList = []
for row in countryData.find_all('tr'):
    
    rowData = []

    rowData = [x.text for x in row.find_all('td')]
    countryDataList.append(rowData)
    
header = countryDataList[0]
data = countryDataList[3:]

# this section will take the header of country information and then split
# last element and update it and add it back to header
total_gross = header[-1].split('/')[0].strip().replace(' ', '_')
total_gross_as_of = total_gross + '_date'
total_gross_as_of

header.pop(-1)
header.extend([total_gross, total_gross_as_of])
header

['Country(click to view weekend breakdown)',
 'Dist.',
 'ReleaseDate',
 'OpeningWknd',
 '% ofTotal',
 'Total_Gross',
 'Total_Gross_date']

In [176]:
import pandas as pd

df = pd.DataFrame(data)
df.columns = header
df

,Country(click to view weekend breakdown),Dist.,ReleaseDate,OpeningWknd,% ofTotal,Total_Gross,Total_Gross_date
0,Argentina,Disney,2/18/16,"$1,886,816",21.6%,"$8,745,000",6/12/16
1,Australia,Disney,3/17/16,"$3,223,798",15.1%,"$21,364,382",7/10/16
2,Austria,Disney,3/3/16,"$698,909",18.7%,"$3,735,647",7/10/16
3,Belgium,Disney,2/10/16,"$1,275,034",28.5%,"$4,467,463",7/10/16
4,Bolivia,Disney,2/18/16,"$124,530",13.7%,"$909,666",4/17/16
5,Brazil,Disney,3/17/16,"$2,574,107",24.2%,"$10,657,941",7/3/16
6,Bulgaria,Forum Films,3/4/16,"$236,098",23.4%,"$1,010,473",7/10/16
7,Chile,Disney,2/18/16,-,-,"$4,266,919",4/17/16
8,China,-,3/4/16,"$23,627,192",10%,"$235,591,000",6/12/16
9,Colombia,Disney,2/18/16,"$1,072,350",20.1%,"$5,330,391",4/17/16
